# Use GTFlow Data as Header and Replace with FCMR Data

In [3]:
%matplotlib qt

import os
import numpy as np
import numpy.matlib
import nibabel as nib
import matplotlib
import matplotlib.pyplot as plt

from __future__ import unicode_literals  # Only for python2.7 and save_as unicode filename
import pydicom
from pydicom.dataset import Dataset, FileDataset, FileMetaDataset
from pydicom.sequence import Sequence

outputDirM = r'E:\Users\tr17\Documents\Projects\PC_Fetal_CMR\Data\dicom_fcmr_4d\dcm\codify_multi_slice\M'
if not os.path.exists(outputDirM):
    os.makedirs(outputDirM)

outputDirR = r'E:\Users\tr17\Documents\Projects\PC_Fetal_CMR\Data\dicom_fcmr_4d\dcm\codify_multi_slice\R'
if not os.path.exists(outputDirR):
    os.makedirs(outputDirR)

outputDirV = []
outputDirV.append(r'E:\Users\tr17\Documents\Projects\PC_Fetal_CMR\Data\dicom_fcmr_4d\dcm\codify_multi_slice\V0')
outputDirV.append(r'E:\Users\tr17\Documents\Projects\PC_Fetal_CMR\Data\dicom_fcmr_4d\dcm\codify_multi_slice\V1')
outputDirV.append(r'E:\Users\tr17\Documents\Projects\PC_Fetal_CMR\Data\dicom_fcmr_4d\dcm\codify_multi_slice\V2')

for i in range(3):
    if not os.path.exists(outputDirV[i]):
        os.makedirs(outputDirV[i])

# Add Hidden Required Tags to pydicom Dictionary

In [4]:
from pydicom.datadict import DicomDictionary, keyword_dict
from pydicom.dataset import Dataset

# Define items as (VR, VM, description, is_retired flag, keyword)
#   Leave is_retired flag blank.
new_dict_items = {
    0x20010010: ('LO', '1', "Private Creator", '', 'PrivateCreator'),
    0x20011008: ('IS', '1', "Phase Number", '', 'PhaseNumber'),
    0x2001100a: ('IS', '1', "Slice Number MR", '', 'SliceNumberMR'),
    0x20011016: ('SS', '1', "Number Of PC Directions", '', 'NumberOfPCDirections'),
    0x20011017: ('SL', '1', "Number Of Phases MR", '', 'NumberOfPhasesMR'),
    0x20011018: ('SL', '1', "Number Of Slices MR", '', 'NumberOfSlicesMR'),
    0x2001101a: ('FL', '3', "PC Velocity", '', 'PCVelocity'),
    0x2001101d: ('IS', '1', "Reconstruction Number MR", '', 'ReconstructionNumberMR'),
    0x00089209: ('CS', '1', "Acquisition Contrast", '', 'AcquisitionContrast'),
    0x00189014: ('CS', '1', "Phase Contrast", '', 'PhaseContrast'),
    0x00189090: ('FD', '3', "Velocity Encoding Direction", '', 'VelocityEncodingDirection'),
    0x00189091: ('FD', '1', "Velocity Encoding Minimum Value", '', 'VelocityEncodingMinimumValue'),
}

# Update the dictionary itself
DicomDictionary.update(new_dict_items)

# Update the reverse mapping from name to tag
new_names_dict = dict([(val[4], tag) for tag, val in
                       new_dict_items.items()])
keyword_dict.update(new_names_dict)

# Load In FCMR Nifty Data

In [5]:
fcmrDir=r'E:\Users\tr17\Documents\Projects\PC_Fetal_CMR\Data\4D_Flow_Paper\fcmr202'

cineVolDir=r'\cine_vol'
cineVolniiFileName=r'\cine_vol.nii.gz'
cineVol_nii = nib.load(fcmrDir+cineVolDir+cineVolniiFileName)
cineVol_img = cineVol_nii.get_fdata()

velVolDir=r'\vel_vol'
velVol0niiFileName=r'\velocity-final-polyCorr-0.nii.gz'
velVol1niiFileName=r'\velocity-final-polyCorr-1.nii.gz'
velVol2niiFileName=r'\velocity-final-polyCorr-2.nii.gz'
velVol0_nii = nib.load(fcmrDir+velVolDir+velVol0niiFileName)
velVol1_nii = nib.load(fcmrDir+velVolDir+velVol1niiFileName)
velVol2_nii = nib.load(fcmrDir+velVolDir+velVol2niiFileName)
velVol0_img = velVol0_nii.get_fdata()
velVol1_img = velVol1_nii.get_fdata()
velVol2_img = velVol2_nii.get_fdata()

venc = 159 # cm/s

print("Shape of cine_vol nifti:", cineVol_img.shape)
print("Shape of vel_vol0 nifti:", velVol0_img.shape)
print("Shape of vel_vol1 nifti:", velVol1_img.shape)
print("Shape of vel_vol2 nifti:", velVol2_img.shape)

if not cineVol_img.shape==velVol0_img.shape:
    print("WARNING: 4D cine_vol and 4D vel_vol volumes are different shapes.")

nX = cineVol_img.shape[0]
nY = cineVol_img.shape[1]
nZ = cineVol_img.shape[2]
nF = cineVol_img.shape[3]

dimX = cineVol_nii.header['pixdim'][1]
dimY = cineVol_nii.header['pixdim'][2]
dimZ = cineVol_nii.header['pixdim'][3]
dimF = cineVol_nii.header['pixdim'][4]

print("pixdim [mm, mm, mm, seconds]:", [dimX, dimY, dimZ, dimF])

c   = np.reshape(cineVol_img, [nX, nY, nZ*nF])
v0  = np.reshape(velVol0_img, [nX, nY, nZ*nF])
v1  = np.reshape(velVol1_img, [nX, nY, nZ*nF])
v2  = np.reshape(velVol2_img, [nX, nY, nZ*nF])

# set background pixels = 0 (-1 in SVRTK)
iBkrd = c==-1
c[iBkrd] = 0; v0[iBkrd] = 0; v1[iBkrd] = 0; v2[iBkrd] = 0
# v0[iBkrd] = -venc; v1[iBkrd] = -venc; v2[iBkrd] = -venc

# convert to same datatype as DICOM
c = c.astype("uint16") 

print("Reshaped size of cine_vol,  c:", c.shape)
print("Reshaped size of vel_vol0, v0:", v0.shape)
print("Reshaped size of vel_vol1, v1:", v1.shape)
print("Reshaped size of vel_vol2, v2:", v2.shape)

# Number of files to create
numInstances = nZ*nF

# View
im2disp = 600
vR = 0.4
fig, ax = plt.subplots(2, 2, sharex='col', sharey='row', figsize=(10,10))
ax[0,0].imshow(c[:,:,im2disp],  cmap=plt.cm.gray)
ax[0,1].imshow(v0[:,:,im2disp], cmap=plt.cm.jet, vmin=-vR, vmax=vR)
ax[1,0].imshow(v1[:,:,im2disp], cmap=plt.cm.jet, vmin=-vR, vmax=vR)
ax[1,1].imshow(v2[:,:,im2disp], cmap=plt.cm.jet, vmin=-vR, vmax=vR)

Shape of cine_vol nifti: (43, 44, 52, 25)
Shape of vel_vol0 nifti: (43, 44, 52, 25)
Shape of vel_vol1 nifti: (43, 44, 52, 25)
Shape of vel_vol2 nifti: (43, 44, 52, 25)
pixdim [mm, mm, mm, seconds]: [1.25, 1.25, 1.25, 0.01714744]
Reshaped size of cine_vol,  c: (43, 44, 1300)
Reshaped size of vel_vol0, v0: (43, 44, 1300)
Reshaped size of vel_vol1, v1: (43, 44, 1300)
Reshaped size of vel_vol2, v2: (43, 44, 1300)


## Velocity Volumes rescaled for DICOMs

In [6]:
RescaleSlope = 1
RescaleIntercept = venc

v0r = v0 * 1e2 * RescaleSlope + RescaleIntercept
v1r = v1 * 1e2 * RescaleSlope + RescaleIntercept
v2r = v2 * 1e2 * RescaleSlope + RescaleIntercept

v0rWindowWidth = round(np.amax(v0r)-np.amin(v0r), 2); v0rWindowCenter = v0rWindowWidth / 2
v1rWindowWidth = round(np.amax(v1r)-np.amin(v1r), 2); v1rWindowCenter = v1rWindowWidth / 2
v2rWindowWidth = round(np.amax(v2r)-np.amin(v2r), 2); v2rWindowCenter = v2rWindowWidth / 2

# convert to same datatype as DICOM
v0r = v0r.astype("uint16")
v1r = v1r.astype("uint16") 
v2r = v2r.astype("uint16") 

# Shift background by venc, so V = 0 with Rescale
v0r[iBkrd] = 0; v1r[iBkrd] = 0; v2r[iBkrd] = 0

# %matplotlib qt
fig, ax = plt.subplots(2, 2, sharex='col', sharey='row', figsize=(10,10))
ax[0,0].imshow(c[:,:,im2disp], cmap=plt.cm.gray)
ax[0,1].imshow(v0r[:,:,im2disp], cmap=plt.cm.gray) #TODO: add in windowing based on mean nonzero signal +/ range
ax[1,0].imshow(v1r[:,:,im2disp], cmap=plt.cm.gray)
ax[1,1].imshow(v2r[:,:,im2disp], cmap=plt.cm.gray)

# Create Arrays to convert from Instance to Frame/Slice Locations

In [7]:
# range from 0 to 657ms because max trigger time in GTFlow data = 657ms.
triggerTimes = np.linspace(0.0, 657.0, num=25)
triggerTimesArray = np.matlib.repmat(triggerTimes, nZ, 1)
triggerTimesArray = triggerTimesArray.ravel()

frameNumbers = np.linspace(1, 25, num=25)
frameNumbersArray = np.matlib.repmat(frameNumbers, nZ, 1)
frameNumbersArray = frameNumbersArray.ravel()

# slice array
sliceIndices = range(1, nZ+1)
sliceIndicesArray = np.repeat(sliceIndices, nF)

# slice locations array
voxelSpacing = 1.25
zLocLast = (voxelSpacing * nZ) - voxelSpacing
sliceLoca = np.linspace(0, zLocLast, num=nZ)
sliceLocaArray = np.repeat(sliceLoca, nF)

# Test working
single_frame_instance = 26
print('Single Frame instance number:', single_frame_instance) # nb: instances count from 1
print('Slice number:', sliceIndicesArray[single_frame_instance-1])
print('Slice location:', sliceLocaArray[single_frame_instance-1])
print('Frame number:', frameNumbersArray[single_frame_instance-1])
print('Trigger time:', triggerTimesArray[single_frame_instance-1])

# Trigger Times plot
plt.figure(figsize=(10,10))
plt.plot(triggerTimesArray,'-r')
plt.xlabel('Dicom instance', fontsize=18)
plt.ylabel('Trigger Time [ms]', fontsize=18)
plt.title('TriggerTimesArray', fontsize=18)
plt.show()

# Slice Indices plot
plt.figure(figsize=(10,10))
plt.plot(sliceIndicesArray,'ob')
plt.xlabel('Dicom instance', fontsize=18)
plt.ylabel('Slice location index', fontsize=18)
plt.title('sliceIndicesArray', fontsize=18)
plt.show()

Single Frame instance number: 26
Slice number: 2
Slice location: 1.25
Frame number: 1.0
Trigger time: 0.0


# Codify Dump - Magnitude
Codify gives a python script to generate the DICOM for a single slice.

In this section, I have commented out the attributes which need to be manually defined or iteratively updated, thus leaving attributes common to all frames/slices

In [8]:
# Coded version of DICOM file 'E:\Users\tr17\Documents\Projects\PC_Fetal_CMR\Data\dicom_fcmr_4d\GTflow_Aorta\10000000\10000001\10000464\100005A3'
# Produced by pydicom codify utility script
from __future__ import unicode_literals  # Only for python2.7 and save_as unicode filename
import pydicom
from pydicom.dataset import Dataset
from pydicom.sequence import Sequence

# File meta info data elements
file_meta = Dataset()
# file_meta.FileMetaInformationGroupLength = 194 ### REQUIRES DEFINITION
file_meta.FileMetaInformationVersion = b'\x00\x01'
file_meta.MediaStorageSOPClassUID = '1.2.840.10008.5.1.4.1.1.4'
# file_meta.MediaStorageSOPInstanceUID = '1.2.40.0.13.1.75591523476291404472265359935487530723' ### REQUIRES DEFINITION
file_meta.TransferSyntaxUID = '1.2.840.10008.1.2'
file_meta.ImplementationClassUID = '1.2.276.0.7230010.3.0.3.6.1'
file_meta.ImplementationVersionName = 'OFFIS_DCMTK_361'

# Main data elements
ds = Dataset()
ds.SpecificCharacterSet = 'ISO_IR 100'
ds.ImageType = ['ORIGINAL', 'PRIMARY', 'M_FFE', 'M', 'FFE']
ds.InstanceCreationDate = '20120821'
ds.InstanceCreationTime = '221700'
ds.InstanceCreatorUID = '1.2.40.0.13.1.203399489339977079628124438700844270739'
ds.SOPClassUID = '1.2.840.10008.5.1.4.1.1.4'
# ds.SOPInstanceUID = '1.2.40.0.13.1.75591523476291404472265359935487530723' ### REQUIRES DEFINITION
ds.StudyDate = '20120821'
ds.SeriesDate = '20120821'
ds.AcquisitionDate = '20120821'
ds.ContentDate = '20120821'
ds.StudyTime = '173207'
ds.SeriesTime = '182511.32000'
ds.AcquisitionTime = '182511.32'
ds.ContentTime = '182511.32'
ds.AccessionNumber = ''
ds.Modality = 'MR'
ds.Manufacturer = 'Philips Medical Systems'
ds.CodeValue = ''
ds.CodingSchemeDesignator = 'DCM'
ds.CodeMeaning = ''

# Procedure Code Sequence
procedure_code_sequence = Sequence()
ds.ProcedureCodeSequence = procedure_code_sequence

# Procedure Code Sequence: Procedure Code 1
procedure_code1 = Dataset()
procedure_code1.CodeValue = 'RA.MRAAOT'
procedure_code1.CodingSchemeDesignator = '99ORBIS'
procedure_code1.CodeMeaning = 'CE-MRA Aorta thorakal'
procedure_code1.ContextGroupExtensionFlag = 'N'
procedure_code_sequence.append(procedure_code1)

ds.OperatorsName = ''
ds.AdmittingDiagnosesDescription = ''
ds.ManufacturerModelName = '*'

# Referenced Performed Procedure Step Sequence
refd_performed_procedure_step_sequence = Sequence()
ds.ReferencedPerformedProcedureStepSequence = refd_performed_procedure_step_sequence

# Referenced Performed Procedure Step Sequence: Referenced Performed Procedure Step 1
refd_performed_procedure_step1 = Dataset()
refd_performed_procedure_step1.InstanceCreationDate = '20120821'
refd_performed_procedure_step1.InstanceCreationTime = '221424'
refd_performed_procedure_step1.InstanceCreatorUID = '1.2.40.0.13.1.203399489339977079628124438700844270739'
refd_performed_procedure_step1.ReferencedSOPClassUID = '1.2.840.10008.3.1.2.3.3'
refd_performed_procedure_step1.ReferencedSOPInstanceUID = '1.3.46.670589.11.17204.5.0.6524.2012082117320696006'
refd_performed_procedure_step1.InstanceNumber = "0"
refd_performed_procedure_step_sequence.append(refd_performed_procedure_step1)


# Referenced Image Sequence
refd_image_sequence = Sequence()
ds.ReferencedImageSequence = refd_image_sequence

# Referenced Image Sequence: Referenced Image 1
refd_image1 = Dataset()
refd_image1.ReferencedSOPClassUID = '1.2.840.10008.5.1.4.1.1.4'
refd_image1.ReferencedSOPInstanceUID = '1.2.40.0.13.1.89078282904346598403696206113943676723'
refd_image_sequence.append(refd_image1)

# Referenced Image Sequence: Referenced Image 2
refd_image2 = Dataset()
refd_image2.ReferencedSOPClassUID = '1.2.840.10008.5.1.4.1.1.4'
refd_image2.ReferencedSOPInstanceUID = '1.2.40.0.13.1.295129673873169057216869911833080985343'
refd_image_sequence.append(refd_image2)

# Referenced Image Sequence: Referenced Image 3
refd_image3 = Dataset()
refd_image3.ReferencedSOPClassUID = '1.2.840.10008.5.1.4.1.1.4'
refd_image3.ReferencedSOPInstanceUID = '1.2.40.0.13.1.37560432539838529536104187971339317428'
refd_image_sequence.append(refd_image3)

ds.PatientName = '-*-'
ds.PatientID = '*'
ds.IssuerOfPatientID = ''
ds.PatientBirthDate = ''
ds.OtherPatientIDs = ''
ds.OtherPatientNames = '*'
ds.PatientMotherBirthName = '*'
ds.PregnancyStatus = 4
ds.ScanningSequence = 'GR'
ds.SequenceVariant = 'SP'
ds.ScanOptions = 'FC'
ds.MRAcquisitionType = '3D'
ds.SequenceName = ''
ds.SliceThickness = "2.6"
ds.RepetitionTime = "3.58299994468688"
ds.EchoTime = "2.288"
ds.NumberOfAverages = "1"
ds.ImagingFrequency = "127.768401"
ds.ImagedNucleus = '1H'
ds.EchoNumbers = "1"
ds.MagneticFieldStrength = "3"
ds.SpacingBetweenSlices = "2.6"
ds.NumberOfPhaseEncodingSteps = "143"
ds.EchoTrainLength = "3"
ds.PercentSampling = "98.4375"
ds.PercentPhaseFieldOfView = "86.4864871376439"
ds.PixelBandwidth = "3284"
ds.SoftwareVersions = ['3.2.1', '3.2.1.0']
ds.ProtocolName = 'WIP Aorta M SENSE'
# ds.TriggerTime = "622" ### REQUIRES DEFINITION
ds.LowRRValue = "632"
ds.HighRRValue = "733"
ds.IntervalsAcquired = "1132"
ds.IntervalsRejected = "20"
ds.HeartRate = "87"
ds.ReconstructionDiameter = "379.999992370605"
ds.ReceiveCoilName = 'SENSE-XL-Torso'
ds.TransmitCoilName = 'B'
ds.AcquisitionMatrix = [0, 148, 143, 0]
ds.InPlanePhaseEncodingDirection = 'ROW'
ds.FlipAngle = "12"
ds.SAR = "0.60249835252761"
ds.dBdt = "56.7453983579957"
ds.PatientPosition = 'HFS'
ds.AcquisitionDuration = 459.6679992675781
ds.DiffusionBValue = 0.0
ds.DiffusionGradientOrientation = [0.0, 0.0, 0.0]
ds.StudyInstanceUID = '1.2.40.0.13.1.333311361771566580913219583914625766216'
ds.SeriesInstanceUID = '1.2.40.0.13.1.286595144572817015845933344548631223145'
ds.StudyID = '513842.201207030'
ds.SeriesNumber = "1006"
ds.AcquisitionNumber = "10"
# ds.InstanceNumber = "319" ### REQUIRES DEFINITION
# ds.ImagePositionPatient = ['-56.040032677094', '-189.81796011867', '225.026188065538'] ### REQUIRES DEFINITION
ds.ImageOrientationPatient = ['0.51319164037704', '0.85772150754928', '-0.0307911429554', '-0.0599991045892', '6.4554493292E-05', '-0.9981984496116']
ds.FrameOfReferenceUID = '1.2.40.0.13.1.168070265634523572089252568290704983898'
ds.TemporalPositionIdentifier = "1"
ds.NumberOfTemporalPositions = "1"
ds.PositionReferenceIndicator = ''
# ds.SliceLocation = "38.9999961150011" ### REQUIRES DEFINITION
ds.SamplesPerPixel = 1
ds.PhotometricInterpretation = 'MONOCHROME2'
# ds.Rows = 192 ### REQUIRES DEFINITION
# ds.Columns = 192 ### REQUIRES DEFINITION
ds.PixelSpacing = ['1.97916662693023', '1.97916662693023']
ds.BitsAllocated = 16
ds.BitsStored = 12
ds.HighBit = 11
ds.PixelRepresentation = 0
ds.WindowCenter = "213.04"
ds.WindowWidth = "370.49"
ds.LossyImageCompression = '00'
ds.RequestingPhysician = '*'
ds.RequestingService = '*'
ds.RequestedProcedureDescription = 'CE-MRA Aorta thorakal'
ds.PerformedStationAETitle = 'ACHIEVA3'
ds.PerformedProcedureStepStartDate = '20120821'
ds.PerformedProcedureStepStartTime = '173207'
ds.PerformedProcedureStepEndDate = '20120821'
ds.PerformedProcedureStepEndTime = '173207'
ds.PerformedProcedureStepID = '398712726'
ds.PerformedProcedureStepDescription = 'CE-MRA Aorta thorakal'

# Performed Protocol Code Sequence
performed_protocol_code_sequence = Sequence()
ds.PerformedProtocolCodeSequence = performed_protocol_code_sequence

# Performed Protocol Code Sequence: Performed Protocol Code 1
performed_protocol_code1 = Dataset()
performed_protocol_code1.CodeValue = 'RA.MRAAOT'
performed_protocol_code1.CodingSchemeDesignator = '99ORBIS'
performed_protocol_code1.CodeMeaning = 'CE-MRA Aorta thorakal'
performed_protocol_code1.ContextGroupExtensionFlag = 'N'
performed_protocol_code_sequence.append(performed_protocol_code1)


# Film Consumption Sequence
film_consumption_sequence = Sequence()
ds.FilmConsumptionSequence = film_consumption_sequence

ds.RequestedProcedureID = '513842.201207030'

# Real World Value Mapping Sequence
real_world_value_mapping_sequence = Sequence()
ds.RealWorldValueMappingSequence = real_world_value_mapping_sequence

# Real World Value Mapping Sequence: Real World Value Mapping 1
real_world_value_mapping1 = Dataset()
real_world_value_mapping1.RealWorldValueIntercept = 0.0
real_world_value_mapping1.RealWorldValueSlope = 4.280830280830281
real_world_value_mapping_sequence.append(real_world_value_mapping1)



# Attributes to Define
Using the code in the previous section, this part defines and updates the attibutes specific to the FCMR data

In [9]:
iFileCtr = 0

for iImage in range(numInstances):
    
    iFileCtr = iFileCtr + 1
    iInst  = iImage + 1
    iSlice = sliceIndicesArray[iInst-1]
    iFrame = frameNumbersArray[iInst-1]
    triggerTime = triggerTimesArray[iInst-1]
    sliceLocation = sliceLocaArray[iInst-1]

#     # Debug
#     print('Instance number:', iInst )
#     print('Slice number:', iSlice)
#     print('Slice location:', sliceLocation)
#     print('Frame number:', iFrame)
#     print('Trigger time:', triggerTime)
    
    randomSOPInstanceUID = pydicom.uid.generate_uid(None)

    # file_meta.FileMetaInformationGroupLength = 194 # leave empty for now 
    
    # Define UIDs - these two are the same within a dicom frame
    file_meta.MediaStorageSOPInstanceUID = randomSOPInstanceUID
    ds.SOPInstanceUID = randomSOPInstanceUID
    
    ### UPDATE Loop Attributes
    ds.TriggerTime = str(triggerTime)
    ds.InstanceNumber = str(iInst)
    ds.ImagePositionPatient = [str(-1),str(-1),str(sliceLocation)] # TODO: update - should give centre coordinate of slice, i.e: -100, -100, -50
    ds.SliceLocation = str(sliceLocation)
    
    ### Phase Contrast Attributes Not Found by Codify
    ds.PrivateCreator = 'Philips Imaging DD 001'
    ds.PhaseNumber = int(iFrame)
    ds.SliceNumberMR = int(iSlice)
    ds.NumberOfPCDirections = 1
    ds.NumberOfSlicesMR = nZ
    ds.NumberOfPhasesMR = nF
    ds.PCVelocity = [0, 0, venc] # Magnitude dicoms have venc as 3rd arg. Not sure if important.
    # ds.ReconstructionNumberMR = 6 # TODO: Not sure if required.
    ds.AcquisitionContrast = 'FLOW_ENCODED'
    ds.PhaseContrast = 'YES'
    # ds.VelocityEncodingDirection = [0.0, 0.0, 0.0] # TODO: Not sure if required.
    ds.VelocityEncodingMinimumValue = 0.0 # TODO: Not sure if required.
    

    ### OVERRIDE Fixed Codify Attributes:
    ds.PatientName = 'fcmr202_Test'
    ds.ProtocolName = 'WIP FCMR M'
    ds.SeriesNumber = "1006"

    ds.Rows = nX
    ds.Columns = nY
    ds.SliceThickness = str(voxelSpacing)
    ds.ImageOrientationPatient = ['1','0','0','0','1','0'] # axial. TODO: update to match Nifti
    ds.PixelSpacing = [str(voxelSpacing), str(voxelSpacing)]
    
    ds.WindowCenter = "1"
    ds.WindowWidth = "0.5"
    real_world_value_mapping1.RealWorldValueIntercept = 0.0
    real_world_value_mapping1.RealWorldValueSlope = 1
    
    # Below essentially lifted from Codify
    ds.PresentationLUTShape = 'IDENTITY'

    ds.PixelData = c[:,:,iImage].tobytes()

    ds.file_meta = file_meta
    ds.is_implicit_VR = True
    ds.is_little_endian = True
    ds.save_as(outputDirM + r'\IM_%04d'%(iFileCtr), write_like_original=False)

    
print('Magnitude DICOM creation complete.')
print('Output directory:', outputDirM)

Magnitude DICOM creation complete.
Output directory: E:\Users\tr17\Documents\Projects\PC_Fetal_CMR\Data\dicom_fcmr_4d\dcm\codify_multi_slice\M


# Codify Dump - Reference Volume

In [10]:
# Coded version of DICOM file '100008C7'
# Produced by pydicom codify utility script
from __future__ import unicode_literals  # Only for python2.7 and save_as unicode filename
import pydicom
from pydicom.dataset import Dataset
from pydicom.sequence import Sequence

# File meta info data elements
file_meta = Dataset()
# file_meta.FileMetaInformationGroupLength = 196 ### REQUIRES DEFINITION
file_meta.FileMetaInformationVersion = b'\x00\x01'
file_meta.MediaStorageSOPClassUID = '1.2.840.10008.5.1.4.1.1.4'
# file_meta.MediaStorageSOPInstanceUID = '1.2.40.0.13.1.319147120590917703017906032097683819796' ### REQUIRES DEFINITION
file_meta.TransferSyntaxUID = '1.2.840.10008.1.2'
file_meta.ImplementationClassUID = '1.2.276.0.7230010.3.0.3.6.1'
file_meta.ImplementationVersionName = 'OFFIS_DCMTK_361'

# Main data elements
ds = Dataset()
ds.SpecificCharacterSet = 'ISO_IR 100'
ds.ImageType = ['ORIGINAL', 'PRIMARY', 'M_FFE', 'M', 'FFE']
ds.InstanceCreationDate = '20120821'
ds.InstanceCreationTime = '205824'
ds.InstanceCreatorUID = '1.2.40.0.13.1.203399489339977079628124438700844270739'
ds.SOPClassUID = '1.2.840.10008.5.1.4.1.1.4'
# ds.SOPInstanceUID = '1.2.40.0.13.1.319147120590917703017906032097683819796' ### REQUIRES DEFINITION
ds.StudyDate = '20120821'
ds.SeriesDate = '20120821'
ds.AcquisitionDate = '20120821'
ds.ContentDate = '20120821'
ds.StudyTime = '173207'
ds.SeriesTime = '182002.67000'
ds.AcquisitionTime = '182002.67'
ds.ContentTime = '182002.67'
ds.AccessionNumber = ''
ds.Modality = 'MR'
ds.Manufacturer = 'Philips Medical Systems'
ds.CodeValue = ''
ds.CodingSchemeDesignator = 'DCM'
ds.CodeMeaning = ''

# Procedure Code Sequence
procedure_code_sequence = Sequence()
ds.ProcedureCodeSequence = procedure_code_sequence

# Procedure Code Sequence: Procedure Code 1
procedure_code1 = Dataset()
procedure_code1.CodeValue = 'RA.MRAAOT'
procedure_code1.CodingSchemeDesignator = '99ORBIS'
procedure_code1.CodeMeaning = 'CE-MRA Aorta thorakal'
procedure_code1.ContextGroupExtensionFlag = 'N'
procedure_code_sequence.append(procedure_code1)

ds.OperatorsName = ''
ds.AdmittingDiagnosesDescription = ''
ds.ManufacturerModelName = '*'

# Referenced Performed Procedure Step Sequence
refd_performed_procedure_step_sequence = Sequence()
ds.ReferencedPerformedProcedureStepSequence = refd_performed_procedure_step_sequence

# Referenced Performed Procedure Step Sequence: Referenced Performed Procedure Step 1
refd_performed_procedure_step1 = Dataset()
refd_performed_procedure_step1.InstanceCreationDate = '20120821'
refd_performed_procedure_step1.InstanceCreationTime = '205814'
refd_performed_procedure_step1.InstanceCreatorUID = '1.2.40.0.13.1.203399489339977079628124438700844270739'
refd_performed_procedure_step1.ReferencedSOPClassUID = '1.2.840.10008.3.1.2.3.3'
refd_performed_procedure_step1.ReferencedSOPInstanceUID = '1.3.46.670589.11.17204.5.0.6524.2012082117320696006'
refd_performed_procedure_step1.InstanceNumber = "0"
refd_performed_procedure_step_sequence.append(refd_performed_procedure_step1)


# Referenced Image Sequence
refd_image_sequence = Sequence()
ds.ReferencedImageSequence = refd_image_sequence

# Referenced Image Sequence: Referenced Image 1
refd_image1 = Dataset()
refd_image1.ReferencedSOPClassUID = '1.2.840.10008.5.1.4.1.1.4'
refd_image1.ReferencedSOPInstanceUID = '1.2.40.0.13.1.252177834452227199512342511739646050769'
refd_image_sequence.append(refd_image1)

# Referenced Image Sequence: Referenced Image 2
refd_image2 = Dataset()
refd_image2.ReferencedSOPClassUID = '1.2.840.10008.5.1.4.1.1.4'
refd_image2.ReferencedSOPInstanceUID = '1.2.40.0.13.1.300968675906340330557095714200932421148'
refd_image_sequence.append(refd_image2)

# Referenced Image Sequence: Referenced Image 3
refd_image3 = Dataset()
refd_image3.ReferencedSOPClassUID = '1.2.840.10008.5.1.4.1.1.4'
refd_image3.ReferencedSOPInstanceUID = '1.2.40.0.13.1.37560432539838529536104187971339317428'
refd_image_sequence.append(refd_image3)

ds.PatientName = '-*-'
ds.PatientID = '*'
ds.IssuerOfPatientID = ''
ds.PatientBirthDate = ''
ds.OtherPatientIDs = ''
ds.OtherPatientNames = '*'
ds.PatientMotherBirthName = '*'
ds.PregnancyStatus = 4
ds.BodyPartExamined = 'HEART'
ds.ScanningSequence = 'GR'
ds.SequenceVariant = 'OSP'
ds.ScanOptions = 'PFF'
ds.MRAcquisitionType = '3D'
ds.SequenceName = ''
ds.SliceThickness = "3.0"
ds.RepetitionTime = "5.11899995803833"
ds.EchoTime = "1.743"
ds.NumberOfAverages = "2.0"
ds.ImagingFrequency = "127.768381999999"
ds.ImagedNucleus = '1H'
ds.EchoNumbers = "1"
ds.MagneticFieldStrength = "3.0"
ds.SpacingBetweenSlices = "1.5"
ds.NumberOfPhaseEncodingSteps = "224"
ds.EchoTrainLength = "1"
ds.PercentSampling = "78.7323150634765"
ds.PercentPhaseFieldOfView = "91.0714259840968"
ds.PixelBandwidth = "434.0"
ds.SoftwareVersions = ['3.2.1', '3.2.1.0']
ds.ProtocolName = 'WIP s3D_HI-RES SENSE'
# ds.TriggerTime = "0.0" ### REQUIRES DEFINITION
ds.LowRRValue = "0"
ds.HighRRValue = "0"
ds.IntervalsAcquired = "0"
ds.IntervalsRejected = "0"
ds.HeartRate = "0"
ds.ReconstructionDiameter = "337.000001907348"
ds.ReceiveCoilName = 'SENSE-XL-Torso'
ds.TransmitCoilName = 'B'
ds.AcquisitionMatrix = [0, 224, 224, 0]
ds.InPlanePhaseEncodingDirection = 'ROW'
ds.FlipAngle = "30.0"
ds.SAR = "0.87467950582504"
ds.dBdt = "41.7317402100092"
ds.PatientPosition = 'HFS'
ds.AcquisitionDuration = 18.56100082397461
ds.DiffusionBValue = 0.0
ds.DiffusionGradientOrientation = [0.0, 0.0, 0.0]
ds.StudyInstanceUID = '1.2.40.0.13.1.333311361771566580913219583914625766216'
ds.SeriesInstanceUID = '1.2.40.0.13.1.120969395829863674500657786028618695061'
ds.StudyID = '513842.201207030'
ds.SeriesNumber = "601"
ds.AcquisitionNumber = "6"
ds.InstanceNumber = "1"
# ds.ImagePositionPatient = [9.36353130843202, -217.04985449268, 211.371105136334] ### REQUIRES DEFINITION
# ds.ImageOrientationPatient = [0.38924595713615, 0.92113387584686, -1.096222591E-11, -0.0004001602064, 0.00016909673286, -0.9999998807907] ### REQUIRES DEFINITION
ds.FrameOfReferenceUID = '1.2.40.0.13.1.168070265634523572089252568290704983898'
ds.TemporalPositionIdentifier = "1"
ds.NumberOfTemporalPositions = "1"
ds.PositionReferenceIndicator = ''
# ds.SliceLocation = "0.0" ### REQUIRES DEFINITION
ds.SamplesPerPixel = 1
ds.PhotometricInterpretation = 'MONOCHROME2'
# ds.Rows = 432 ### REQUIRES DEFINITION
# ds.Columns = 432 ### REQUIRES DEFINITION
# ds.PixelSpacing = [0.78009259700775, 0.78009259700775] ### REQUIRES DEFINITION
ds.BitsAllocated = 16
ds.BitsStored = 12
ds.HighBit = 11
ds.PixelRepresentation = 0
ds.WindowCenter = "640.46"
ds.WindowWidth = "1113.84"
ds.LossyImageCompression = '00'
ds.RequestingPhysician = '*'
ds.RequestingService = '*'
ds.RequestedProcedureDescription = 'CE-MRA Aorta thorakal'
ds.PerformedStationAETitle = 'ACHIEVA3'
ds.PerformedProcedureStepStartDate = '20120821'
ds.PerformedProcedureStepStartTime = '173207'
ds.PerformedProcedureStepEndDate = '20120821'
ds.PerformedProcedureStepEndTime = '173207'
ds.PerformedProcedureStepID = '398712726'
ds.PerformedProcedureStepDescription = 'CE-MRA Aorta thorakal'

# Performed Protocol Code Sequence
performed_protocol_code_sequence = Sequence()
ds.PerformedProtocolCodeSequence = performed_protocol_code_sequence

# Performed Protocol Code Sequence: Performed Protocol Code 1
performed_protocol_code1 = Dataset()
performed_protocol_code1.CodeValue = 'RA.MRAAOT'
performed_protocol_code1.CodingSchemeDesignator = '99ORBIS'
performed_protocol_code1.CodeMeaning = 'CE-MRA Aorta thorakal'
performed_protocol_code1.ContextGroupExtensionFlag = 'N'
performed_protocol_code_sequence.append(performed_protocol_code1)


# Film Consumption Sequence
film_consumption_sequence = Sequence()
ds.FilmConsumptionSequence = film_consumption_sequence

ds.RequestedProcedureID = '513842.201207030'

# Real World Value Mapping Sequence
real_world_value_mapping_sequence = Sequence()
ds.RealWorldValueMappingSequence = real_world_value_mapping_sequence

# Real World Value Mapping Sequence: Real World Value Mapping 1
real_world_value_mapping1 = Dataset()
real_world_value_mapping1.RealWorldValueIntercept = 0.0
real_world_value_mapping1.RealWorldValueSlope = 1.221001221001221
real_world_value_mapping_sequence.append(real_world_value_mapping1)

# Attributes to Define

In [11]:
# Create Reference Volume Using 1st Frame of cine_vol

for iImage in range(0,numInstances,nF):
    
    iInst  = iImage + 1
    iSlice = sliceIndicesArray[iInst-1]
    iFrame = frameNumbersArray[iInst-1]
    triggerTime = triggerTimesArray[iInst-1]
    sliceLocation = sliceLocaArray[iInst-1]

    # # Debug
    # print('Instance number:', iInst )
    # print('Slice number:', iSlice)
    # print('Slice location:', sliceLocation)
    # print('Frame number:', iFrame)
    # print('Trigger time:', triggerTime)
    
    randomSOPInstanceUID = pydicom.uid.generate_uid(None)
    
    # Define UIDs - these two are the same within a dicom frame
    file_meta.MediaStorageSOPInstanceUID = randomSOPInstanceUID
    ds.SOPInstanceUID = randomSOPInstanceUID
    
    ### UPDATE Loop Attributes
    ds.TriggerTime = str(triggerTime)
    ds.InstanceNumber = str(int(1 + (iInst - 1)/nF)) # Altered to reflect one frame
    ds.ImagePositionPatient = [str(-1),str(-1),str(sliceLocation)] # TODO: update - should give centre coordinate of slice, i.e: -100, -100, -50
    ds.SliceLocation = str(sliceLocation)
    
    ### Phase Contrast Attributes Not Found by Codify
    ds.PrivateCreator = 'Philips Imaging DD 001'
    ds.PhaseNumber = int(iFrame)
    ds.SliceNumberMR = int(iSlice)
    ds.NumberOfPCDirections = 0
    ds.NumberOfSlicesMR = nZ
    ds.NumberOfPhasesMR = nF
    ds.PCVelocity = [0, 0, 0] # Magnitude dicoms have venc as 3rd arg. Not sure if important.
    # ds.ReconstructionNumberMR = 1 # TODO: Not sure if required.
    ds.AcquisitionContrast = 'bFFE'
    ds.PhaseContrast = 'NO'
    # ds.VelocityEncodingDirection = [0.0, 0.0, 0.0] # TODO: Not sure if required.
    ds.VelocityEncodingMinimumValue = 0.0 # TODO: Not sure if required.
    

    ### OVERRIDE Fixed Codify Attributes:
    ds.PatientName = 'fcmr202_Test'
    ds.ProtocolName = 'WIP FCMR 3D bFFE'
    ds.SeriesNumber = "601"

    ds.Rows = nX
    ds.Columns = nY
    ds.SliceThickness = str(voxelSpacing)
    ds.ImageOrientationPatient = ['1','0','0','0','1','0'] # axial. TODO: update to match Nifti
    ds.PixelSpacing = [str(voxelSpacing), str(voxelSpacing)]
    
    ds.WindowCenter = "1"
    ds.WindowWidth = "0.5"
    real_world_value_mapping1.RealWorldValueIntercept = 0.0
    real_world_value_mapping1.RealWorldValueSlope = 1
    
    # Below essentially lifted from Codify
    ds.PresentationLUTShape = 'IDENTITY'

    ds.PixelData = c[:,:,iImage].tobytes()

    ds.file_meta = file_meta
    ds.is_implicit_VR = True
    ds.is_little_endian = True
    ds.save_as(outputDirR + r'\IM_%04d'%(ds.InstanceNumber), write_like_original=False)

    
print('Magnitude DICOM creation complete.')
print('Output directory:', outputDirR)

Magnitude DICOM creation complete.
Output directory: E:\Users\tr17\Documents\Projects\PC_Fetal_CMR\Data\dicom_fcmr_4d\dcm\codify_multi_slice\R


# Codify Dump - Velocity

Same as before, but for one a velocity DICOM

NB: currently not using corresponding instances of magnitude/velocity DICOMs

In [12]:
# Coded version of DICOM file '10000003'
# Produced by pydicom codify utility script
from __future__ import unicode_literals  # Only for python2.7 and save_as unicode filename
import pydicom
from pydicom.dataset import Dataset
from pydicom.sequence import Sequence

# File meta info data elements
file_meta = Dataset()
# file_meta.FileMetaInformationGroupLength = 194 ### REQUIRES DEFINITION
file_meta.FileMetaInformationVersion = b'\x00\x01'
file_meta.MediaStorageSOPClassUID = '1.2.840.10008.5.1.4.1.1.4'
# file_meta.MediaStorageSOPInstanceUID = '1.2.40.0.13.1.7844659339164932257841219351304201515' ### REQUIRES DEFINITION
file_meta.TransferSyntaxUID = '1.2.840.10008.1.2'
file_meta.ImplementationClassUID = '1.2.276.0.7230010.3.0.3.6.1'
file_meta.ImplementationVersionName = 'OFFIS_DCMTK_361'

# Main data elements
ds = Dataset()
ds.SpecificCharacterSet = 'ISO_IR 100'
ds.ImageType = ['ORIGINAL', 'PRIMARY', 'VELOCITY MAP', 'P', 'PCA']
ds.InstanceCreationDate = '20120821'
# ds.InstanceCreationTime = '221515' ### REQUIRES DEFINITION
ds.InstanceCreatorUID = '1.2.40.0.13.1.203399489339977079628124438700844270739'
ds.SOPClassUID = '1.2.840.10008.5.1.4.1.1.4'
# ds.SOPInstanceUID = '1.2.40.0.13.1.7844659339164932257841219351304201515' ### REQUIRES DEFINITION
ds.StudyDate = '20120821'
ds.SeriesDate = '20120821'
ds.AcquisitionDate = '20120821'
ds.ContentDate = '20120821'
ds.StudyTime = '173207'
ds.SeriesTime = '182511.32000'
ds.AcquisitionTime = '182511.32'
ds.ContentTime = '182511.32'
ds.AccessionNumber = ''
ds.Modality = 'MR'
ds.Manufacturer = 'Philips Medical Systems'
ds.CodeValue = ''
ds.CodingSchemeDesignator = 'DCM'
ds.CodeMeaning = ''

# Procedure Code Sequence
procedure_code_sequence = Sequence()
ds.ProcedureCodeSequence = procedure_code_sequence

# Procedure Code Sequence: Procedure Code 1
procedure_code1 = Dataset()
procedure_code1.CodeValue = 'RA.MRAAOT'
procedure_code1.CodingSchemeDesignator = '99ORBIS'
procedure_code1.CodeMeaning = 'CE-MRA Aorta thorakal'
procedure_code1.ContextGroupExtensionFlag = 'N'
procedure_code_sequence.append(procedure_code1)

ds.OperatorsName = ''
ds.AdmittingDiagnosesDescription = ''
ds.ManufacturerModelName = '*'

# Referenced Performed Procedure Step Sequence
refd_performed_procedure_step_sequence = Sequence()
ds.ReferencedPerformedProcedureStepSequence = refd_performed_procedure_step_sequence

# Referenced Performed Procedure Step Sequence: Referenced Performed Procedure Step 1
refd_performed_procedure_step1 = Dataset()
refd_performed_procedure_step1.InstanceCreationDate = '20120821'
refd_performed_procedure_step1.InstanceCreationTime = '221424'
refd_performed_procedure_step1.InstanceCreatorUID = '1.2.40.0.13.1.203399489339977079628124438700844270739'
refd_performed_procedure_step1.ReferencedSOPClassUID = '1.2.840.10008.3.1.2.3.3'
refd_performed_procedure_step1.ReferencedSOPInstanceUID = '1.3.46.670589.11.17204.5.0.6524.2012082117320696006'
refd_performed_procedure_step1.InstanceNumber = "0"
refd_performed_procedure_step_sequence.append(refd_performed_procedure_step1)


# Referenced Image Sequence
refd_image_sequence = Sequence()
ds.ReferencedImageSequence = refd_image_sequence

# Referenced Image Sequence: Referenced Image 1
refd_image1 = Dataset()
refd_image1.ReferencedSOPClassUID = '1.2.840.10008.5.1.4.1.1.4'
refd_image1.ReferencedSOPInstanceUID = '1.2.40.0.13.1.89078282904346598403696206113943676723'
refd_image_sequence.append(refd_image1)

# Referenced Image Sequence: Referenced Image 2
refd_image2 = Dataset()
refd_image2.ReferencedSOPClassUID = '1.2.840.10008.5.1.4.1.1.4'
refd_image2.ReferencedSOPInstanceUID = '1.2.40.0.13.1.295129673873169057216869911833080985343'
refd_image_sequence.append(refd_image2)

# Referenced Image Sequence: Referenced Image 3
refd_image3 = Dataset()
refd_image3.ReferencedSOPClassUID = '1.2.840.10008.5.1.4.1.1.4'
refd_image3.ReferencedSOPInstanceUID = '1.2.40.0.13.1.37560432539838529536104187971339317428'
refd_image_sequence.append(refd_image3)

ds.PatientName = '-*-'
ds.PatientID = '*'
ds.IssuerOfPatientID = ''
ds.PatientBirthDate = ''
ds.OtherPatientIDs = ''
ds.OtherPatientNames = '*'
ds.PatientMotherBirthName = '*'
ds.PregnancyStatus = 4
ds.ScanningSequence = 'GR'
ds.SequenceVariant = 'SP'
ds.ScanOptions = 'FC'
ds.MRAcquisitionType = '3D'
ds.SequenceName = ''
# ds.SliceThickness = "2.6" ### REQUIRES DEFINITION
ds.RepetitionTime = "3.58299994468688"
ds.EchoTime = "2.288"
ds.NumberOfAverages = "1.0"
ds.ImagingFrequency = "127.768401"
ds.ImagedNucleus = '1H'
ds.EchoNumbers = "1"
ds.MagneticFieldStrength = "3.0"
ds.SpacingBetweenSlices = "2.6"
ds.NumberOfPhaseEncodingSteps = "143"
ds.EchoTrainLength = "3"
ds.PercentSampling = "98.4375"
ds.PercentPhaseFieldOfView = "86.4864871376439"
ds.PixelBandwidth = "3284.0"
ds.SoftwareVersions = ['3.2.1', '3.2.1.0']
# ds.ProtocolName = 'WIP Aorta AP SENSE' ### REQUIRES DEFINITION
# ds.TriggerTime = "0.0" ### REQUIRES DEFINITION
ds.LowRRValue = "632"
ds.HighRRValue = "733"
ds.IntervalsAcquired = "1132"
ds.IntervalsRejected = "20"
ds.HeartRate = "87"
ds.ReconstructionDiameter = "379.999992370605"
ds.ReceiveCoilName = 'SENSE-XL-Torso'
ds.TransmitCoilName = 'B'
ds.AcquisitionMatrix = [0, 148, 143, 0]
ds.InPlanePhaseEncodingDirection = 'ROW'
ds.FlipAngle = "12.0"
ds.SAR = "0.60249835252761"
ds.dBdt = "56.7453983579957"
ds.PatientPosition = 'HFS'
ds.AcquisitionDuration = 459.6679992675781
ds.DiffusionBValue = 0.0
ds.DiffusionGradientOrientation = [0.0, 0.0, 0.0]
ds.StudyInstanceUID = '1.2.40.0.13.1.333311361771566580913219583914625766216'
# ds.SeriesInstanceUID = '1.2.40.0.13.1.9736333416408446660135208989318902663' ### REQUIRES DEFINITION
ds.StudyID = '513842.201207030'
# ds.SeriesNumber = "1004" ### REQUIRES DEFINITION
ds.AcquisitionNumber = "10"
# ds.InstanceNumber = "1" ### REQUIRES DEFINITION
# ds.ImagePositionPatient = [-22.649262718598, -209.86846671047, 223.017864396105] ### REQUIRES DEFINITION
ds.ImageOrientationPatient = [0.51319164037704, 0.85772150754928, -0.0307911429554, -0.0599991045892, 6.4554493292E-05, -0.9981984496116] 
ds.FrameOfReferenceUID = '1.2.40.0.13.1.168070265634523572089252568290704983898'
ds.TemporalPositionIdentifier = "1"
ds.NumberOfTemporalPositions = "1"
ds.PositionReferenceIndicator = ''
# ds.SliceLocation = "0.0" ### REQUIRES DEFINITION
ds.SamplesPerPixel = 1
ds.PhotometricInterpretation = 'MONOCHROME2'
# ds.Rows = 192 ### REQUIRES DEFINITION
# ds.Columns = 192 ### REQUIRES DEFINITION
ds.PixelSpacing = [1.97916662693023, 1.97916662693023]
ds.BitsAllocated = 16
ds.BitsStored = 12
ds.HighBit = 11
ds.PixelRepresentation = 0
ds.WindowCenter = "2047.5"
ds.WindowWidth = "4095.0"
ds.LossyImageCompression = '00'
ds.RequestingPhysician = '*'
ds.RequestingService = '*'
ds.RequestedProcedureDescription = 'CE-MRA Aorta thorakal'
ds.PerformedStationAETitle = 'ACHIEVA3'
ds.PerformedProcedureStepStartDate = '20120821'
ds.PerformedProcedureStepStartTime = '173207'
ds.PerformedProcedureStepEndDate = '20120821'
ds.PerformedProcedureStepEndTime = '173207'
ds.PerformedProcedureStepID = '398712726'
ds.PerformedProcedureStepDescription = 'CE-MRA Aorta thorakal'

# Performed Protocol Code Sequence
performed_protocol_code_sequence = Sequence()
ds.PerformedProtocolCodeSequence = performed_protocol_code_sequence

# Performed Protocol Code Sequence: Performed Protocol Code 1
performed_protocol_code1 = Dataset()
performed_protocol_code1.CodeValue = 'RA.MRAAOT'
performed_protocol_code1.CodingSchemeDesignator = '99ORBIS'
performed_protocol_code1.CodeMeaning = 'CE-MRA Aorta thorakal'
performed_protocol_code1.ContextGroupExtensionFlag = 'N'
performed_protocol_code_sequence.append(performed_protocol_code1)


# Film Consumption Sequence
film_consumption_sequence = Sequence()
ds.FilmConsumptionSequence = film_consumption_sequence

ds.RequestedProcedureID = '513842.201207030'

### SECTION BELOW NEEDS DEFINITION

# Real World Value Mapping Sequence
real_world_value_mapping_sequence = Sequence()
ds.RealWorldValueMappingSequence = real_world_value_mapping_sequence

# # Real World Value Mapping Sequence: Real World Value Mapping 1
# real_world_value_mapping1 = Dataset()
# real_world_value_mapping1.RealWorldValueIntercept = -180.0
# real_world_value_mapping1.RealWorldValueSlope = 0.08791208791208792
# real_world_value_mapping_sequence.append(real_world_value_mapping1)

# Attributes to Define - Velocity

In [13]:
# Real World Value Mapping Sequence: Real World Value Mapping 1
real_world_value_mapping1 = Dataset()
real_world_value_mapping1.RealWorldValueIntercept = venc
real_world_value_mapping1.RealWorldValueSlope = 1
real_world_value_mapping_sequence.append(real_world_value_mapping1)

for iVelVol in range(3):

    # Attributes Different Between Velocity Volumes

    if iVelVol==0:
        ds.ProtocolName = 'WIP FCMR Vel0'
        ds.InstanceCreationTime = '221424'
        ds.SeriesNumber = "1003"
        ds.WindowWidth = str(v0rWindowWidth)
        ds.WindowCenter = str(v0rWindowCenter)
        ds.PCVelocity = [venc, 0, 0]
        # ds.ReconstructionNumberMR = 3 # TODO: Not sure if required.
    elif iVelVol==1:
        ds.ProtocolName = 'WIP FCMR Vel1'
        ds.InstanceCreationTime = '221515'
        ds.SeriesNumber = "1004"
        ds.WindowWidth = str(v1rWindowWidth)
        ds.WindowCenter = str(v1rWindowCenter)
        ds.PCVelocity = [0, venc, 0]
        # ds.ReconstructionNumberMR = 4 # TODO: Not sure if required.
    elif iVelVol==2:
        ds.ProtocolName = 'WIP FCMR Vel2'
        ds.InstanceCreationTime = '221603'
        ds.SeriesNumber = "1005"
        ds.WindowWidth = str(v2rWindowWidth)
        ds.WindowCenter = str(v2rWindowCenter)
        ds.PCVelocity = [0, 0, venc]
        # ds.ReconstructionNumberMR = 5 # TODO: Not sure if required.

    ds.PatientName = 'fcmr202_Test'
    ds.SeriesInstanceUID = pydicom.uid.generate_uid(None)
    ds.ImageType = ['ORIGINAL', 'PRIMARY', 'VELOCITY MAP', 'P', 'PCA']


    for iImage in range(numInstances):
        
        iFileCtr = iFileCtr + 1
        iInst  = iImage + 1
        iSlice = sliceIndicesArray[iInst-1]
        iFrame = frameNumbersArray[iInst-1]
        triggerTime = triggerTimesArray[iInst-1]
        sliceLocation = sliceLocaArray[iInst-1]

    #     # Debug
    #     print('Instance number:', iInst )
    #     print('Slice number:', iSlice)
    #     print('Slice location:', sliceLocation)
    #     print('Frame number:', iFrame)
    #     print('Trigger time:', triggerTime)
        
        randomSOPInstanceUID = pydicom.uid.generate_uid(None)

        # file_meta.FileMetaInformationGroupLength = 194 # leave empty for now 
        
        # Define UIDs - these two are the same within a dicom frame
        file_meta.MediaStorageSOPInstanceUID = randomSOPInstanceUID
        ds.SOPInstanceUID = randomSOPInstanceUID
        
        ### UPDATE Loop Attributes
        ds.TriggerTime = str(triggerTime)
        ds.InstanceNumber = str(iInst)
        ds.ImagePositionPatient = [str(-1),str(-1),str(sliceLocation)] # TODO: update - should give centre coordinate of slice, i.e: -100, -100, -50
        ds.SliceLocation = str(sliceLocation)
        
        ### Phase Contrast Attributes Not Found by Codify
        ds.PrivateCreator = 'Philips Imaging DD 001'
        ds.PhaseNumber = int(iFrame)
        ds.SliceNumberMR = int(iSlice)
        ds.NumberOfPCDirections = 1
        ds.NumberOfSlicesMR = nZ
        ds.NumberOfPhasesMR = nF
        ds.AcquisitionContrast = 'FLOW_ENCODED'
        ds.PhaseContrast = 'YES'
        # ds.VelocityEncodingDirection = [0.0, 0.0, 0.0] # TODO: Not sure if required.
        ds.VelocityEncodingMinimumValue = 0.0 # TODO: Not sure if required.


        ### OVERRIDE Fixed Codify Attributes:
        ds.Rows = nX
        ds.Columns = nY
        ds.SliceThickness = str(voxelSpacing)
        ds.ImageOrientationPatient = ['1','0','0','0','1','0'] # axial. TODO: update to match Nifti
        ds.PixelSpacing = [str(voxelSpacing), str(voxelSpacing)]
        
        # Below essentially lifted from Codify
        ds.PresentationLUTShape = 'IDENTITY'

        if iVelVol==0:
            ds.PixelData = v0r[:,:,iImage].tobytes()
        elif iVelVol==1:
            ds.PixelData = v1r[:,:,iImage].tobytes()
        elif iVelVol==2:
            ds.PixelData = v2r[:,:,iImage].tobytes()

        ds.file_meta = file_meta
        ds.is_implicit_VR = True
        ds.is_little_endian = True
        ds.save_as(outputDirV[iVelVol] + r'\IM_%04d'%(iFileCtr), write_like_original=False)

    
print('Velocity DICOM creation complete.')
print('Output directories:')
print(outputDirV[0]); print(outputDirV[1]); print(outputDirV[2])

Velocity DICOM creation complete.
Output directories:
E:\Users\tr17\Documents\Projects\PC_Fetal_CMR\Data\dicom_fcmr_4d\dcm\codify_multi_slice\V0
E:\Users\tr17\Documents\Projects\PC_Fetal_CMR\Data\dicom_fcmr_4d\dcm\codify_multi_slice\V1
E:\Users\tr17\Documents\Projects\PC_Fetal_CMR\Data\dicom_fcmr_4d\dcm\codify_multi_slice\V2


# Open DICOM to debug

In [14]:
# os.chdir(outputDirM)
os.chdir(outputDirV[2])
# os.listdir()

# early frame
dcmToCheck = pydicom.dcmread('IM_3901')

# with open('dcm_mr000.txt', 'w') as f:
#     print(dcmToCheck, file=f)

print(dcmToCheck)


# # later frame
# dcmToCheck_slicerRepaired = pydicom.dcmread('mr025.dcm')

# with open('dcm_mr025.txt', 'w') as f:
#     print(dcmToCheck_slicerRepaired, file=f)

Dataset.file_meta -------------------------------
(0002, 0000) File Meta Information Group Length  UL: 186
(0002, 0001) File Meta Information Version       OB: b'\x00\x01'
(0002, 0002) Media Storage SOP Class UID         UI: MR Image Storage
(0002, 0003) Media Storage SOP Instance UID      UI: 2.25.190826704712111702106049318917590751006
(0002, 0010) Transfer Syntax UID                 UI: Implicit VR Little Endian
(0002, 0012) Implementation Class UID            UI: 1.2.276.0.7230010.3.0.3.6.1
(0002, 0013) Implementation Version Name         SH: 'OFFIS_DCMTK_361'
-------------------------------------------------
(0008, 0005) Specific Character Set              CS: 'ISO_IR 100'
(0008, 0008) Image Type                          CS: ['ORIGINAL', 'PRIMARY', 'VELOCITY MAP', 'P', 'PCA']
(0008, 0012) Instance Creation Date              DA: '20120821'
(0008, 0013) Instance Creation Time              TM: '221603'
(0008, 0014) Instance Creator UID                UI: 1.2.40.0.13.1.203399489339977